In [20]:
import numpy as np
import math
import random
import copy

In [21]:
# size of problem
n = 12
l = 6
t_max = 10000

# drs is a distance between locations r and s
D = np.array([[0, 1, 2, 3, 1, 2, 3, 4, 2, 3, 4, 5], 
              [1, 0, 1, 2, 2, 1, 2, 3, 3, 2, 3, 4],
              [2, 1, 0, 1, 3, 2, 1, 2, 4, 3, 2, 3],
              [3, 2, 1, 0, 4, 3, 2, 1, 5, 4, 3, 2],
              [1, 2, 3, 4, 0, 1, 2, 3, 1, 2, 3, 4],
              [2, 1, 2, 3, 1, 0, 1, 2, 2, 1, 2, 3],
              [3, 2, 1, 2, 2, 1, 0, 1, 3, 2, 1, 2],
              [4, 3, 2, 1, 3, 2, 1, 0, 4, 3, 2, 1],
              [2, 3, 4, 5, 1, 2, 3, 4, 0, 1, 2, 3],
              [3, 2, 3, 4, 2, 1, 2, 3, 1, 0, 1, 2],
              [4, 3, 2, 3, 3, 2, 1, 2, 2, 1, 0, 1],
              [5, 4, 3, 2, 4, 3, 2, 1, 3, 2, 1, 0]])

# wij is the flow between facilities i and j
W = np.array([[0, 5, 2, 4, 1, 0, 0, 6, 2, 1, 1, 1],
              [5, 0, 3, 0, 2, 2, 2, 0, 4, 5, 0, 0],
              [2, 3, 0, 0, 0, 0, 0, 5, 5, 2, 2, 2],
              [4, 0, 0, 0, 5, 2, 2, 10, 0, 0, 5, 5],
              [1, 2, 0, 5, 0, 10, 0, 0, 0, 5, 1, 1],
              [0, 2, 0, 2, 10, 0, 5, 1, 1, 5, 4, 0],
              [0, 2, 0, 2, 0, 5, 0, 10, 5, 2, 3, 3],
              [6, 0, 5, 10, 0, 1, 10, 0, 0, 0, 5, 0],
              [2, 4, 5, 0, 0, 1, 5, 0, 0, 0, 10, 10],
              [1, 5, 2, 0, 5, 5, 2, 0, 0, 0, 5, 0],
              [1, 0, 2, 5, 1, 4, 3, 5, 10, 5, 0, 2],
              [1, 0, 2, 5, 1, 0, 3, 0, 10, 0, 2, 0]])

# a state will be of form : [12,11,10,9,8,7,6,5,4,3,2,1]
# it means that:
# facility 12 is in location 1
# facility 11 is in location 2
# facility 10 is in location 3

# ...

# facility s[i] is in location i

tabu_list = np.zeros((12,12))
print(tabu_list)
# represents the forbidden transitions, known as the short term memory
# each row represents a facility and the collum the position
# if tabu_list[i][j] = l -> facility i can't be placed in position j for the next l moves 
# if tabu_list[i][j] = 0 -> transition is legal
print()
diversification_meca = np.zeros((12,12))
print(diversification_meca)
# represents the transitions we want to prioritise, known as the long term memory
# if diversification_meca[i][j] = n^2 -> facility i must be placed in position j in the next move

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [22]:
# functions 

# taken from tp0 -> used for generation of random initial state
def balancedDice(N):
    return math.floor(random.random() * N)

# create a random array with all the facilities placed in random locations
def generate_random_state(): 
    facilities = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
    s = []
    while len(facilities) > 0:
        i = balancedDice(len(facilities))
        s.append(facilities[i])
        facilities.pop(i)
    s = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
    return s

# simple fitness computation
def compute_fitness(s):
    total_fit = 0
    for i in range(len(s)):
        for j in range(i,len(s)):
            distance = D[i][j]
            weight = W[s[i]][s[j]]
            total_fit = total_fit + distance * weight
    return total_fit * 2

# checks forced transitions -> diversication mecanism -> if a forced transition happens to be a forbiden transition:
# it won't be selected as a transition
# for example we have a forced transition to place facility 3 in village 2
# but the facility in villace 2, is forbidden from being placed in the village in which facility 3 is 
# -> we can't make the switch -> mast want until the move is legal
def check_forced_transitions(s):
    transitions = []
    u = n*n
    for i in range(diversification_meca.shape[0]):
        for j in range(diversification_meca.shape[1]):
            if diversification_meca[i][j] == u:
                transition = (s.index(i), j)
                if tabu_list[s[j]][transition[0]] == 0:
                    print(s[j])
                    print(transition[0])
                    transitions.append(transition)
    return transitions

# for all transitions, we check if they are legal -> according to the tabu list
def check_transitions(s, transitions):
    post_tr = copy.deepcopy(transitions)
    to_delete = []
    i = 0
    while i < len(transitions):
        city_from = transitions[i][0] # transitions[i][0] -> city from
        city_to = transitions[i][1] # transitions[i][1] -> city to
        facility_in_city_from = s[transitions[i][0]] # s[transitions[i][0]] -> facility in city from
        facility_in_city_to = s[transitions[i][1]] # s[transitions[i][1]] -> facility in city to
        # if either tabu_list[facility_from_to][city_to_from] > 0 -> locked transition -> we delete it 
        if tabu_list[facility_in_city_from][city_to] > 0 or tabu_list[facility_in_city_to][city_from] > 0 :
            to_delete.append(i)
        i = i + 1
    i = len(to_delete) - 1
    while i > -1:
        post_tr.pop(to_delete[i])
        i = i - 1
    return post_tr

# compute all transitions -> checks if there are forced and forbidden transitions
def compute_transitions(s):
    transitions = check_forced_transitions(s)
    if len(transitions) > 0:
        return transitions
    transitions = []
    for i in range(len(s)):
        for j in range(i+1,len(s)):
            transitions.append((i,j))
    transitions = check_transitions(s,transitions)
    return transitions

# execute all the transitions in the array transitions from the state s
def execute_transitions(s, transitions):
    neighborhood = []
    for transition in transitions:
        s_nei = copy.copy(s)
        s_nei[transition[0]], s_nei[transition[1]] = s_nei[transition[1]], s_nei[transition[0]]
        neighborhood.append(s_nei)
    return neighborhood

# update tabu list as well as diversification mechanism list
def update_tabu_diversification_meca(s, transition):
    for i in range(tabu_list.shape[0]):
        for j in range(tabu_list.shape[1]):
            if tabu_list[i][j] > 0:
                tabu_list[i][j] = tabu_list[i][j] - 1
            if diversification_meca[i][j] < n*n:
                diversification_meca[i][j] = diversification_meca[i][j] + 1               
    tabu_list[s[transition[0]]][transition[0]] = l
    tabu_list[s[transition[1]]][transition[1]] = l
    diversification_meca[s[transition[0]]][transition[0]] = 0
    diversification_meca[s[transition[1]]][transition[1]] = 0

# compute fitness but FASTER -> CODE GO BRRRRR!
def faster_compute_fitness(s, fit, transition):
    c_fit = copy.copy(fit)
    transition_fitness = []
    for i in range(tabu_list.shape[0]):
        c_fit = c_fit - D[transition[0]][i] * W[s[transition[0]]][s[i]]
        c_fit = c_fit - D[transition[1]][i] * W[s[transition[1]]][s[i]]
    s = execute_transitions(s, [transition])[0]
    for i in range(tabu_list.shape[0]):
        c_fit = c_fit + D[transition[1]][i] * W[s[transition[1]]][s[i]]
        c_fit = c_fit + D[transition[0]][i] * W[s[transition[0]]][s[i]]    
    return c_fit


In [23]:
initial_state = generate_random_state()
fitness_initial_state = compute_fitness(initial_state)
transitions_legal = compute_transitions(initial_state)
neighbors = execute_transitions(initial_state, transitions_legal)

In [24]:
def global_function(initial_state, fitness_initial_state):
    state = initial_state
    fitness = fitness_initial_state

    all_time_fitness = []
    all_time_fitness.append(fitness)

    best_state = initial_state
    best_fitness = fitness_initial_state

    for i in range(t_max):
        transitions_legal = compute_transitions(state)
        transitions_fit = [faster_compute_fitness(state, fitness, transitions_legal[j]) for j in range(len(transitions_legal))]
        new_state = execute_transitions(state, [transitions_legal[np.argmin(transitions_fit)]])[0]
        new_fit = transitions_fit[np.argmin(transitions_fit)]
        all_time_fitness.append(new_fit)
        if new_fit < best_fitness:
            best_fitness = new_fit
            best_state = new_state 
        update_tabu_diversification_meca(new_state, transitions_legal[np.argmin(transitions_fit)])
    return best_state, best_fitness, all_time_fitness


In [25]:
states = []
fitness = []
all_time_list = []

for i in range(10):
    initial_state = generate_random_state()
    fitness_initial_state = compute_fitness(initial_state)
    s, fit, all_time = global_function(initial_state, fitness_initial_state)
    states.append(s)
    fitness.append(fit)
    all_time_list.append(all_time)
    print("######################################             -> ", i)

0
0
1
0
3
0
4
0
5
0
6
0
7
0
8
0
9
0
10
0
11
0
0
1
1
1
2
1
3
1
5
1
6
1
7
1
8
1
9
1
10
1
11
1
1
2
2
2
3
2
5
2
6
2
7
2
8
2
9
2
10
2
11
2
0
3
1
3
2
3
3
3
4
3
5
3
6
3
9
3
10
3
11
3
0
4
3
4
4
4
5
4
6
4
7
4
8
4
9
4
10
4
11
4
0
5
1
5
2
5
3
5
4
5
5
5
6
5
7
5
9
5
10
5
11
5
0
6
1
6
2
6
3
6
4
6
5
6
6
6
7
6
8
6
9
6
10
6
11
6
0
7
1
7
2
7
4
7
5
7
6
7
7
7
8
7
9
7
10
7
11
7
0
8
1
8
2
8
4
8
6
8
7
8
8
8
10
8
11
8
0
9
1
9
2
9
3
9
4
9
5
9
6
9
7
9
9
9
10
9
11
9
0
10
1
10
2
10
3
10
4
10
5
10
6
10
7
10
8
10
9
10
10
10
11
10
0
11
1
11
2
11
3
11
4
11
5
11
6
11
7
11
8
11
9
11
10
11
11
11
0
0
3
0
4
0
5
0
6
0
7
0
8
0
9
0
10
0
11
0
1
1
2
1
3
1
5
1
6
1
7
1
8
1
9
1
10
1
11
1
1
2
2
2
3
2
5
2
6
2
7
2
8
2
9
2
10
2
11
2
0
3
1
3
2
3
3
3
4
3
5
3
6
3
9
3
10
3
11
3
0
4
3
4
4
4
5
4
6
4
7
4
8
4
9
4
10
4
11
4
0
5
1
5
2
5
3
5
4
5
5
5
6
5
7
5
9
5
10
5
11
5
0
6
1
6
2
6
3
6
4
6
5
6
6
6
7
6
8
6
9
6
10
6
11
6
0
7
1
7
2
7
4
7
5
7
6
7
7
7
8
7
9
7
10
7
11
7
0
8
1
8
2
8
4
8
6
8
7
8
8
8
10
8
11
8
0
9
1
9
2
9
3
9
4
9
5
9
6
9
7
9
9
9
10
9
1

5
1
5
2
5
3
5
4
5
6
5
7
5
9
5
10
5
11
5
0
6
1
6
2
6
3
6
4
6
5
6
7
6
8
6
9
6
10
6
11
6
0
7
1
7
4
7
5
7
6
7
8
7
9
7
10
7
11
7
0
8
1
8
4
8
6
8
7
8
11
8
0
9
1
9
3
9
4
9
5
9
6
9
7
9
10
9
11
9
0
10
1
10
2
10
3
10
4
10
5
10
6
10
7
10
9
10
1
11
3
11
4
11
5
11
6
11
7
11
8
11
9
11
3
0
4
0
5
0
6
0
7
0
8
0
9
0
10
0
2
1
3
1
5
1
6
1
7
1
8
1
9
1
10
1
11
1
1
2
5
2
6
2
10
2
0
3
1
3
4
3
5
3
6
3
9
3
10
3
11
3
0
4
3
4
6
4
7
4
8
4
9
4
10
4
11
4
0
5
1
5
2
5
3
5
6
5
7
5
9
5
10
5
11
5
0
6
1
6
2
6
3
6
4
6
5
6
7
6
8
6
9
6
10
6
11
6
0
7
1
7
4
7
5
7
6
7
8
7
9
7
10
7
11
7
0
8
1
8
4
8
6
8
7
8
11
8
0
9
1
9
3
9
4
9
5
9
6
9
7
9
10
9
11
9
0
10
1
10
2
10
3
10
4
10
5
10
6
10
7
10
9
10
1
11
3
11
4
11
5
11
6
11
7
11
8
11
9
11
3
0
4
0
5
0
6
0
7
0
8
0
9
0
10
0
2
1
3
1
5
1
6
1
7
1
8
1
9
1
10
1
11
1
1
2
5
2
6
2
10
2
0
3
1
3
4
3
5
3
6
3
9
3
10
3
11
3
0
4
3
4
6
4
7
4
8
4
9
4
10
4
11
4
0
5
1
5
2
5
3
5
6
5
7
5
10
5
11
5
0
6
1
6
2
6
3
6
4
6
5
6
7
6
8
6
9
6
10
6
11
6
0
7
1
7
4
7
5
7
6
7
8
7
9
7
10
7
11
7
0
8
1
8
4
8
6
8
7
8
11
8
0
9

3
10
5
10
7
10
1
11
5
11
7
11
3
0
6
0
7
0
8
0
10
0
3
1
6
1
7
1
10
1
11
1
0
3
1
3
4
3
5
3
9
3
10
3
3
4
7
4
3
5
7
5
10
5
11
5
0
6
1
6
0
7
1
7
4
7
5
7
8
7
9
7
10
7
11
7
0
8
7
8
3
9
7
9
0
10
1
10
3
10
5
10
7
10
1
11
5
11
7
11
3
0
6
0
7
0
8
0
10
0
3
1
6
1
7
1
10
1
11
1
0
3
1
3
4
3
5
3
10
3
3
4
7
4
3
5
7
5
10
5
11
5
0
6
1
6
0
7
1
7
4
7
5
7
8
7
9
7
10
7
11
7
0
8
7
8
7
9
0
10
1
10
3
10
5
10
7
10
1
11
5
11
7
11
3
0
6
0
7
0
8
0
10
0
3
1
6
1
7
1
10
1
11
1
0
3
1
3
4
3
5
3
10
3
3
4
7
4
3
5
7
5
10
5
11
5
0
6
1
6
0
7
1
7
4
7
5
7
9
7
10
7
11
7
0
8
7
9
0
10
1
10
3
10
5
10
7
10
1
11
5
11
7
11
3
0
6
0
7
0
8
0
10
0
3
1
6
1
7
1
10
1
11
1
0
3
1
3
4
3
5
3
10
3
3
4
7
4
3
5
7
5
11
5
0
6
1
6
0
7
1
7
4
7
5
7
9
7
10
7
11
7
0
8
7
9
0
10
1
10
3
10
7
10
1
11
5
11
7
11
3
0
6
0
7
0
8
0
10
0
3
1
6
1
7
1
11
1
0
3
1
3
4
3
5
3
10
3
3
4
7
4
3
5
7
5
11
5
0
6
1
6
0
7
1
7
4
7
5
7
9
7
10
7
11
7
0
8
7
9
0
10
3
10
7
10
1
11
5
11
7
11
6
0
7
0
8
0
10
0
3
1
6
1
7
1
11
1
1
3
4
3
5
3
10
3
3
4
7
4
3
5
7
5
11
5
0
6
1
6
0
7
1
7
4
7
5
7


10
11
0
0
11
9
2
2
9
0
0
1
1
2
2
3
3
4
4
5
5
6
6
7
7
8
8
9
9
10
10
11
11
11
2
2
11
3
2
2
3
11
10
10
11
5
4
4
5
9
5
5
9
7
6
6
7
6
3
3
6
9
4
4
9
11
4
4
11
11
6
6
11
11
8
8
11
9
0
0
9
8
1
1
8
10
6
6
10
8
6
6
8
4
0
0
4
9
6
6
9
10
4
4
10
5
0
0
5
9
1
1
9
6
2
2
6
10
9
9
10
5
2
2
5
10
2
2
10
6
4
4
6
8
4
4
8
2
1
1
2
11
3
3
11
11
9
9
11
5
1
1
5
6
5
5
6
9
3
3
9
8
7
7
8
10
5
5
10
10
1
1
10
3
0
0
3
10
0
0
10
11
5
5
11
6
0
0
6
11
7
7
11
11
1
1
11
7
1
1
7
7
4
4
7
8
0
0
8
7
5
5
7
10
7
7
10
6
1
1
6
3
1
1
3
5
3
3
5
10
3
3
10
4
3
3
4
9
7
7
9
7
0
0
7
1
0
0
1
7
2
2
7
8
2
2
8
10
8
8
10
11
0
0
11
9
2
2
9
0
0
1
1
2
2
3
3
4
4
5
5
6
6
7
7
8
8
9
9
10
10
11
11
11
2
2
11
3
2
2
3
11
10
10
11
5
4
4
5
9
5
5
9
7
6
6
7
6
3
3
6
9
4
4
9
11
4
4
11
11
6
6
11
11
8
8
11
9
0
0
9
8
1
1
8
10
6
6
10
8
6
6
8
4
0
0
4
9
6
6
9
10
4
4
10
5
0
0
5
9
1
1
9
6
2
2
6
10
9
9
10
5
2
2
5
10
2
2
10
6
4
4
6
8
4
4
8
2
1
1
2
11
3
3
11
11
9
9
11
5
1
1
5
6
5
5
6
9
3
3
9
8
7
7
8
10
5
5
10
10
1
1
10
3
0
0
3
10
0
0
10
11
5
5
11
6
0
0
6
11
7
7
11
11
1


0
0
1
1
2
2
3
3
4
4
5
5
6
6
7
7
8
8
9
9
10
10
11
11
11
2
2
11
3
2
2
3
11
10
10
11
5
4
4
5
9
5
5
9
7
6
6
7
6
3
3
6
9
4
4
9
11
4
4
11
11
6
6
11
11
8
8
11
9
0
0
9
8
1
1
8
10
6
6
10
8
6
6
8
4
0
0
4
9
6
6
9
10
4
4
10
5
0
0
5
9
1
1
9
6
2
2
6
10
9
9
10
5
2
2
5
10
2
2
10
6
4
4
6
8
4
4
8
2
1
1
2
11
3
3
11
11
9
9
11
5
1
1
5
6
5
5
6
9
3
3
9
8
7
7
8
10
5
5
10
10
1
1
10
3
0
0
3
10
0
0
10
11
5
5
11
6
0
0
6
11
7
7
11
11
1
1
11
7
1
1
7
7
4
4
7
8
0
0
8
7
5
5
7
10
7
7
10
6
1
1
6
3
1
1
3
5
3
3
5
10
3
3
10
4
3
3
4
9
7
7
9
7
0
0
7
1
0
0
1
7
2
2
7
8
2
2
8
10
8
8
10
11
0
0
11
9
2
2
9
0
0
1
1
2
2
3
3
4
4
5
5
6
6
7
7
8
8
9
9
10
10
11
11
11
2
2
11
3
2
2
3
11
10
10
11
5
4
4
5
9
5
5
9
7
6
6
7
6
3
3
6
9
4
4
9
11
4
4
11
11
6
6
11
11
8
8
11
9
0
0
9
8
1
1
8
10
6
6
10
8
6
6
8
4
0
0
4
9
6
6
9
10
4
4
10
5
0
0
5
9
1
1
9
6
2
2
6
10
9
9
10
5
2
2
5
10
2
2
10
6
4
4
6
8
4
4
8
2
1
1
2
11
3
3
11
11
9
9
11
5
1
1
5
6
5
5
6
9
3
3
9
8
7
7
8
10
5
5
10
10
1
1
10
3
0
0
3
10
0
0
10
11
5
5
11
6
0
0
6
11
7
7
11
11
1
1
11
7
1
1
7
7
4
4
7


...

10
11
0
0
11
9
2
2
9
0
0
1
1
2
2
3
3
4
4
5
5
6
6
7
7
8
8
9
9
10
10
11
11
11
2
2
11
3
2
2
3
11
10
10
11
5
4
4
5
9
5
5
9
7
6
6
7
6
3
3
6
9
4
4
9
11
4
4
11
11
6
6
11
11
8
8
11
9
0
0
9
8
1
1
8
10
6
6
10
8
6
6
8
4
0
0
4
9
6
6
9
10
4
4
10
5
0
0
5
9
1
1
9
6
2
2
6
10
9
9
10
5
2
2
5
10
2
2
10
6
4
4
6
8
4
4
8
2
1
1
2
11
3
3
11
11
9
9
11
5
1
1
5
6
5
5
6
9
3
3
9
8
7
7
8
10
5
5
10
10
1
1
10
3
0
0
3
10
0
0
10
11
5
5
11
6
0
0
6
11
7
7
11
11
1
1
11
7
1
1
7
7
4
4
7
8
0
0
8
7
5
5
7
10
7
7
10
6
1
1
6
3
1
1
3
5
3
3
5
10
3
3
10
4
3
3
4
9
7
7
9
7
0
0
7
1
0
0
1
7
2
2
7
8
2
2
8
10
8
8
10
11
0
0
11
9
2
2
9
0
0
1
1
2
2
3
3
4
4
5
5
6
6
7
7
8
8
9
9
10
10
11
11
11
2
2
11
3
2
2
3
11
10
10
11
5
4
4
5
9
5
5
9
7
6
6
7
6
3
3
6
9
4
4
9
11
4
4
11
11
6
6
11
11
8
8
11
9
0
0
9
8
1
1
8
10
6
6
10
8
6
6
8
4
0
0
4
9
6
6
9
10
4
4
10
5
0
0
5
9
1
1
9
6
2
2
6
10
9
9
10
5
2
2
5
10
2
2
10
6
4
4
6
8
4
4
8
2
1
1
2
11
3
3
11
11
9
9
11
5
1
1
5
6
5
5
6
9
3
3
9
8
7
7
8
10
5
5
10
10
1
1
10
3
0
0
3
10
0
0
10
11
5
5
11
6
0
0
6
11
7
7
11
11
1


1
0
0
1
7
2
2
7
8
2
2
8
10
8
8
10
11
0
0
11
9
2
2
9
0
0
1
1
2
2
3
3
4
4
5
5
6
6
7
7
8
8
9
9
10
10
11
11
11
2
2
11
3
2
2
3
11
10
10
11
5
4
4
5
9
5
5
9
7
6
6
7
6
3
3
6
9
4
4
9
11
4
4
11
11
6
6
11
11
8
8
11
9
0
0
9
8
1
1
8
10
6
6
10
8
6
6
8
4
0
0
4
9
6
6
9
10
4
4
10
5
0
0
5
9
1
1
9
6
2
2
6
10
9
9
10
5
2
2
5
10
2
2
10
6
4
4
6
8
4
4
8
2
1
1
2
11
3
3
11
11
9
9
11
5
1
1
5
6
5
5
6
9
3
3
9
8
7
7
8
10
5
5
10
10
1
1
10
3
0
0
3
10
0
0
10
11
5
5
11
6
0
0
6
11
7
7
11
11
1
1
11
7
1
1
7
7
4
4
7
8
0
0
8
7
5
5
7
10
7
7
10
6
1
1
6
3
1
1
3
5
3
3
5
10
3
3
10
4
3
3
4
9
7
7
9
7
0
0
7
1
0
0
1
7
2
2
7
8
2
2
8
10
8
8
10
11
0
0
11
9
2
2
9
0
0
1
1
2
2
3
3
4
4
5
5
6
6
7
7
8
8
9
9
10
10
11
11
11
2
2
11
3
2
2
3
11
10
10
11
5
4
4
5
9
5
5
9
7
6
6
7
6
3
3
6
9
4
4
9
11
4
4
11
11
6
6
11
11
8
8
11
9
0
0
9
8
1
1
8
10
6
6
10
8
6
6
8
4
0
0
4
9
6
6
9
10
4
4
10
5
0
0
5
9
1
1
9
6
2
2
6
10
9
9
10
5
2
2
5
10
2
2
10
6
4
4
6
8
4
4
8
2
1
1
2
11
3
3
11
11
9
9
11
5
1
1
5
6
5
5
6
9
3
3
9
8
7
7
8
10
5
5
10
10
1
1
10
3
0
0
3
10
0
0
10
11

3
3
4
4
5
5
6
6
7
7
8
8
9
9
10
10
11
11
11
2
2
11
3
2
2
3
11
10
10
11
5
4
4
5
9
5
5
9
7
6
6
7
6
3
3
6
9
4
4
9
11
4
4
11
11
6
6
11
11
8
8
11
9
0
0
9
8
1
1
8
10
6
6
10
8
6
6
8
4
0
0
4
9
6
6
9
10
4
4
10
5
0
0
5
9
1
1
9
6
2
2
6
10
9
9
10
5
2
2
5
10
2
2
10
6
4
4
6
8
4
4
8
2
1
1
2
11
3
3
11
11
9
9
11
5
1
1
5
6
5
5
6
9
3
3
9
8
7
7
8
10
5
5
10
10
1
1
10
3
0
0
3
10
0
0
10
11
5
5
11
6
0
0
6
11
7
7
11
11
1
1
11
7
1
1
7
7
4
4
7
8
0
0
8
7
5
5
7
10
7
7
10
6
1
1
6
3
1
1
3
5
3
3
5
10
3
3
10
4
3
3
4
9
7
7
9
7
0
0
7
1
0
0
1
7
2
2
7
8
2
2
8
10
8
8
10
11
0
0
11
9
2
2
9
0
0
1
1
2
2
3
3
4
4
5
5
6
6
7
7
8
8
9
9
10
10
11
11
11
2
2
11
3
2
2
3
11
10
10
11
5
4
4
5
9
5
5
9
7
6
6
7
6
3
3
6
9
4
4
9
11
4
4
11
11
6
6
11
11
8
8
11
9
0
0
9
8
1
1
8
10
6
6
10
8
6
6
8
4
0
0
4
9
6
6
9
10
4
4
10
5
0
0
5
9
1
1
9
6
2
2
6
10
9
9
10
5
2
2
5
10
2
2
10
6
4
4
6
8
4
4
8
2
1
1
2
11
3
3
11
11
9
9
11
5
1
1
5
6
5
5
6
9
3
3
9
8
7
7
8
10
5
5
10
10
1
1
10
3
0
0
3
10
0
0
10
11
5
5
11
6
0
0
6
11
7
7
11
11
1
1
11
7
1
1
7
7
4
4
7
8
0
0
8
7
5


1
1
2
2
3
3
4
4
5
5
6
6
7
7
8
8
9
9
10
10
11
11
11
2
2
11
3
2
2
3
11
10
10
11
5
4
4
5
9
5
5
9
7
6
6
7
6
3
3
6
9
4
4
9
11
4
4
11
11
6
6
11
11
8
8
11
9
0
0
9
8
1
1
8
10
6
6
10
8
6
6
8
4
0
0
4
9
6
6
9
10
4
4
10
5
0
0
5
9
1
1
9
6
2
2
6
10
9
9
10
5
2
2
5
10
2
2
10
6
4
4
6
8
4
4
8
2
1
1
2
11
3
3
11
11
9
9
11
5
1
1
5
6
5
5
6
9
3
3
9
8
7
7
8
10
5
5
10
10
1
1
10
3
0
0
3
10
0
0
10
11
5
5
11
6
0
0
6
11
7
7
11
11
1
1
11
7
1
1
7
7
4
4
7
8
0
0
8
7
5
5
7
10
7
7
10
6
1
1
6
3
1
1
3
5
3
3
5
10
3
3
10
4
3
3
4
9
7
7
9
7
0
0
7
1
0
0
1
7
2
2
7
8
2
2
8
10
8
8
10
11
0
0
11
9
2
2
9
0
0
1
1
2
2
3
3
4
4
5
5
6
6
7
7
8
8
9
9
10
10
11
11
11
2
2
11
3
2
2
3
11
10
10
11
5
4
4
5
9
5
5
9
7
6
6
7
6
3
3
6
9
4
4
9
11
4
4
11
11
6
6
11
11
8
8
11
9
0
0
9
8
1
1
8
10
6
6
10
8
6
6
8
4
0
0
4
9
6
6
9
10
4
4
10
5
0
0
5
9
1
1
9
6
2
2
6
10
9
9
10
5
2
2
5
10
2
2
10
6
4
4
6
8
4
4
8
2
1
1
2
11
3
3
11
11
9
9
11
5
1
1
5
6
5
5
6
9
3
3
9
8
7
7
8
10
5
5
10
10
1
1
10
3
0
0
3
10
0
0
10
11
5
5
11
6
0
0
6
11
7
7
11
11
1
1
11
7
1
1
7
7
4
4
7
8
0


2
8
10
8
8
10
11
0
0
11
9
2
2
9
0
0
1
1
2
2
3
3
4
4
5
5
6
6
7
7
8
8
9
9
10
10
11
11
11
2
2
11
3
2
2
3
11
10
10
11
5
4
4
5
9
5
5
9
7
6
6
7
6
3
3
6
9
4
4
9
11
4
4
11
11
6
6
11
11
8
8
11
9
0
0
9
8
1
1
8
10
6
6
10
8
6
6
8
4
0
0
4
9
6
6
9
10
4
4
10
5
0
0
5
9
1
1
9
6
2
2
6
10
9
9
10
5
2
2
5
10
2
2
10
6
4
4
6
8
4
4
8
2
1
1
2
11
3
3
11
11
9
9
11
5
1
1
5
6
5
5
6
9
3
3
9
8
7
7
8
10
5
5
10
10
1
1
10
3
0
0
3
10
0
0
10
11
5
5
11
6
0
0
6
11
7
7
11
11
1
1
11
7
1
1
7
7
4
4
7
8
0
0
8
7
5
5
7
10
7
7
10
6
1
1
6
3
1
1
3
5
3
3
5
10
3
3
10
4
3
3
4
9
7
7
9
7
0
0
7
1
0
0
1
7
2
2
7
8
2
2
8
10
8
8
10
11
0
0
11
9
2
2
9
0
0
1
1
2
2
3
3
4
4
5
5
6
6
7
7
8
8
9
9
10
10
11
11
11
2
2
11
3
2
2
3
11
10
10
11
5
4
4
5
9
5
5
9
7
6
6
7
6
3
3
6
9
4
4
9
11
4
4
11
11
6
6
11
11
8
8
11
9
0
0
9
8
1
1
8
10
6
6
10
8
6
6
8
4
0
0
4
9
6
6
9
10
4
4
10
5
0
0
5
9
1
1
9
6
2
2
6
10
9
9
10
5
2
2
5
10
2
2
10
6
4
4
6
8
4
4
8
2
1
1
2
11
3
3
11
11
9
9
11
5
1
1
5
6
5
5
6
9
3
3
9
8
7
7
8
10
5
5
10
10
1
1
10
3
0
0
3
10
0
0
10
11
5
5
11
6
0
0
6
11
7

In [26]:
"""
results without tabu 

states = [[1, 9, 8, 11, 2, 10, 4, 5, 0, 7, 6, 3], [5, 11, 8, 0, 10, 7, 2, 9, 4, 3, 6, 1], [10, 8, 11, 0, 2, 6, 1, 7, 3, 4, 5, 9], [4, 10, 2, 1, 7, 11, 8, 9, 3, 6, 5, 0], [2, 5, 0, 11, 1, 6, 4, 8, 7, 9, 10, 3], [0, 5, 4, 8, 6, 1, 10, 11, 7, 3, 9, 2], [1, 9, 4, 5, 11, 8, 10, 6, 0, 3, 2, 7], [3, 7, 6, 9, 0, 2, 8, 10, 11, 5, 1, 4], [4, 2, 10, 5, 11, 8, 9, 1, 3, 0, 7, 6], [4, 1, 0, 3, 5, 8, 2, 11, 10, 9, 6, 7]]
fitness = [335, 374, 388, 382, 384, 357, 321, 369, 355, 349]

#### 
tmax = 10k
STATE - FIT - MEAN - MAX/MIN
-> l = 1
state-> [0, 1, 2, 3, 4, 5, 6, 7, 9, 8, 10, 11]  || fit ->  704  || avg ->  723.2370762923707  || np.max/np.min ->  776  /  704
state-> [0, 1, 2, 3, 4, 5, 6, 7, 9, 8, 10, 11]  || fit ->  704  || avg ->  723.2381761823817  || np.max/np.min ->  776  /  704
state-> [0, 1, 2, 3, 4, 5, 6, 7, 9, 8, 10, 11]  || fit ->  704  || avg ->  723.2391760823917  || np.max/np.min ->  776  /  704
state-> [0, 1, 2, 3, 4, 5, 6, 7, 9, 8, 10, 11]  || fit ->  704  || avg ->  723.2410758924108  || np.max/np.min ->  776  /  704
state-> [0, 1, 2, 3, 4, 5, 6, 7, 9, 8, 10, 11]  || fit ->  704  || avg ->  723.2410758924108  || np.max/np.min ->  776  /  704
state-> [0, 1, 2, 3, 4, 5, 6, 7, 9, 8, 10, 11]  || fit ->  704  || avg ->  723.2423757624238  || np.max/np.min ->  776  /  704
state-> [0, 1, 2, 3, 4, 5, 6, 7, 9, 8, 10, 11]  || fit ->  704  || avg ->  723.2446755324468  || np.max/np.min ->  776  /  704
state-> [0, 1, 2, 3, 4, 5, 6, 7, 9, 8, 10, 11]  || fit ->  704  || avg ->  723.2472752724727  || np.max/np.min ->  776  /  704
state-> [0, 1, 2, 3, 4, 5, 6, 7, 9, 8, 10, 11]  || fit ->  704  || avg ->  723.2494750524947  || np.max/np.min ->  776  /  704
state-> [0, 1, 2, 3, 4, 5, 6, 7, 9, 8, 10, 11]  || fit ->  704  || avg ->  723.2486751324867  || np.max/np.min ->  776  /  704

-> l = 6
state-> [2, 8, 10, 3, 0, 1, 11, 7, 5, 4, 9, 6]  || fit ->  356  || avg ->  385.4552272386381  || np.max/np.min ->  449  /  356
state-> [10, 1, 5, 3, 0, 11, 7, 6, 8, 2, 9, 4]  || fit ->  398  || avg ->  424.32273386330684  || np.max/np.min ->  500  /  398
state-> [8, 7, 10, 1, 2, 11, 9, 5, 0, 3, 6, 4]  || fit ->  370  || avg ->  405.20358982050897  || np.max/np.min ->  456  /  370
state-> [5, 9, 11, 6, 4, 8, 3, 10, 1, 2, 0, 7]  || fit ->  337  || avg ->  370.122393880306  || np.max/np.min ->  427  /  337
state-> [9, 4, 3, 5, 8, 1, 10, 0, 11, 2, 6, 7]  || fit ->  352  || avg ->  377.0340482975851  || np.max/np.min ->  438  /  352
state-> [10, 9, 6, 5, 2, 3, 4, 1, 8, 11, 0, 7]  || fit ->  388  || avg ->  418.8167591620419  || np.max/np.min ->  470  /  388
state-> [8, 5, 2, 1, 3, 11, 4, 9, 10, 6, 7, 0]  || fit ->  368  || avg ->  386.930403479826  || np.max/np.min ->  450  /  368
state-> [8, 0, 9, 1, 6, 2, 7, 4, 11, 10, 3, 5]  || fit ->  349  || avg ->  394.82405879706016  || np.max/np.min ->  461  /  349
state-> [9, 2, 11, 3, 6, 4, 8, 10, 1, 5, 7, 0]  || fit ->  376  || avg ->  400.60021998900055  || np.max/np.min ->  460  /  376
state-> [6, 5, 2, 1, 7, 11, 0, 10, 3, 4, 8, 9]  || fit ->  379  || avg ->  402.0042497875106  || np.max/np.min ->  465  /  379

-> l = 10
state-> [4, 9, 5, 1, 10, 0, 6, 8, 3, 7, 11, 2]  || fit ->  330  || avg ->  363.5823708814559  || np.max/np.min ->  419  /  330
state-> [10, 7, 0, 4, 8, 2, 1, 9, 11, 6, 5, 3]  || fit ->  363  || avg ->  395.5055247237638  || np.max/np.min ->  457  /  363
state-> [10, 6, 7, 8, 1, 3, 0, 2, 9, 4, 5, 11]  || fit ->  356  || avg ->  390.047197640118  || np.max/np.min ->  441  /  356
state-> [0, 7, 2, 3, 10, 4, 6, 11, 9, 5, 1, 8]  || fit ->  365  || avg ->  387.21578921053947  || np.max/np.min ->  444  /  365
state-> [9, 4, 3, 5, 10, 1, 6, 7, 11, 8, 0, 2]  || fit ->  351  || avg ->  372.6381180940953  || np.max/np.min ->  431  /  351
state-> [2, 9, 7, 6, 1, 4, 5, 3, 8, 10, 11, 0]  || fit ->  361  || avg ->  392.61041947902606  || np.max/np.min ->  433  /  361
state-> [10, 5, 6, 4, 8, 11, 7, 3, 9, 0, 2, 1]  || fit ->  365  || avg ->  384.40157992100393  || np.max/np.min ->  427  /  365
state-> [1, 11, 9, 4, 10, 0, 8, 5, 3, 7, 6, 2]  || fit ->  370  || avg ->  392.7169141542923  || np.max/np.min ->  463  /  370
state-> [10, 0, 11, 3, 8, 7, 6, 5, 2, 9, 1, 4]  || fit ->  347  || avg ->  373.6062196890156  || np.max/np.min ->  432  /  347
state-> [5, 9, 4, 10, 6, 11, 7, 0, 1, 2, 3, 8]  || fit ->  380  || avg ->  399.2298385080746  || np.max/np.min ->  448  /  380
"""

'\nresults without tabu \n\nstates = [[1, 9, 8, 11, 2, 10, 4, 5, 0, 7, 6, 3], [5, 11, 8, 0, 10, 7, 2, 9, 4, 3, 6, 1], [10, 8, 11, 0, 2, 6, 1, 7, 3, 4, 5, 9], [4, 10, 2, 1, 7, 11, 8, 9, 3, 6, 5, 0], [2, 5, 0, 11, 1, 6, 4, 8, 7, 9, 10, 3], [0, 5, 4, 8, 6, 1, 10, 11, 7, 3, 9, 2], [1, 9, 4, 5, 11, 8, 10, 6, 0, 3, 2, 7], [3, 7, 6, 9, 0, 2, 8, 10, 11, 5, 1, 4], [4, 2, 10, 5, 11, 8, 9, 1, 3, 0, 7, 6], [4, 1, 0, 3, 5, 8, 2, 11, 10, 9, 6, 7]]\nfitness = [335, 374, 388, 382, 384, 357, 321, 369, 355, 349]\n\n#### \ntmax = 10k\nSTATE - FIT - MEAN - MAX/MIN\n-> l = 1\nstate-> [0, 1, 2, 3, 4, 5, 6, 7, 9, 8, 10, 11]  || fit ->  704  || avg ->  723.2370762923707  || np.max/np.min ->  776  /  704\nstate-> [0, 1, 2, 3, 4, 5, 6, 7, 9, 8, 10, 11]  || fit ->  704  || avg ->  723.2381761823817  || np.max/np.min ->  776  /  704\nstate-> [0, 1, 2, 3, 4, 5, 6, 7, 9, 8, 10, 11]  || fit ->  704  || avg ->  723.2391760823917  || np.max/np.min ->  776  /  704\nstate-> [0, 1, 2, 3, 4, 5, 6, 7, 9, 8, 10, 11]  || fit

In [27]:
for i in range(len(states)):
    print("state->", states[i], " || fit -> ",fitness[i], " || avg -> ", np.sum(all_time_list[i])/len(all_time_list[i]), " || np.max/np.min -> ", np.max(all_time_list[i]), " / ", np.min(all_time_list[i]))


state-> [0, 1, 2, 3, 4, 5, 6, 7, 9, 8, 10, 11]  || fit ->  704  || avg ->  725.1590840915908  || np.max/np.min ->  776  /  704
state-> [0, 1, 2, 3, 4, 5, 6, 7, 9, 8, 10, 11]  || fit ->  704  || avg ->  725.3432656734327  || np.max/np.min ->  776  /  704
state-> [0, 1, 2, 3, 4, 5, 6, 7, 9, 8, 10, 11]  || fit ->  704  || avg ->  725.3444655534447  || np.max/np.min ->  776  /  704
state-> [0, 1, 2, 3, 4, 5, 6, 7, 9, 8, 10, 11]  || fit ->  704  || avg ->  725.3444655534447  || np.max/np.min ->  776  /  704
state-> [0, 1, 2, 3, 4, 5, 6, 7, 9, 8, 10, 11]  || fit ->  704  || avg ->  725.3436656334367  || np.max/np.min ->  776  /  704
state-> [0, 1, 2, 3, 4, 5, 6, 7, 9, 8, 10, 11]  || fit ->  704  || avg ->  725.3442655734426  || np.max/np.min ->  776  /  704
state-> [0, 1, 2, 3, 4, 5, 6, 7, 9, 8, 10, 11]  || fit ->  704  || avg ->  725.3444655534447  || np.max/np.min ->  776  /  704
state-> [0, 1, 2, 3, 4, 5, 6, 7, 9, 8, 10, 11]  || fit ->  704  || avg ->  725.3446655334467  || np.max/np.min 